In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the path to the folder containing ClusterAndPredict
module_path = os.path.abspath(os.path.join('..', '/workspaces/FactChecker-V3'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
from ClusterAndPredict.ClusterAndPredict import ClusterAndPredict
from Testing.DataLoader import DataLoader
from Testing.ParameterCreator import ParameterCreator
from Clustering.Helpers.Visualizer import Visualizer

2024-10-04 20:28:41.181414: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-04 20:28:42.228731: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-04 20:28:42.551155: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-04 20:28:43.118293: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-04 20:28:43.272098: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-04 20:28:44.286632: I tensorflow/core/platform/cpu_feature_gu

In [ ]:
params = ParameterCreator().get_parameters()
results = []
cluster_dfs = []
print("Number of experiments to run: ", len(params))
for param in params:
	percentage = 0.75
	data_loader = DataLoader(percentage, True, param['random_seed'])
	use_only_card = param['use_only_CARD']
	size_of_dataset = param['size_of_dataset']
	del param['size_of_dataset']
	del param['use_only_CARD']

	train_df, test_df = data_loader.create_train_test_df(True, True, True, True, size_of_dataset)
	folds = data_loader.create_cv_folds(train_df, test_df)

	for train_df, test_df in folds:
		print("run")
		clf = ClusterAndPredict(**param, train_df=train_df)
		clf.fit(test_df['Text'].tolist(), test_df['Numerical Rating'].tolist())
		# Print best parameters
		best_estimator = clf
		score = best_estimator.score([], [])
		print(best_estimator.get_all_performance_metrics())
		object_output = best_estimator.get_all_performance_metrics()
		cluster_df = object_output['cluster_df']
		cluster_dfs.append(cluster_df)
		output = {
			'percentage': percentage,
			'score': score,
			'accuracy': best_estimator.get_accuracy(),
			'was_supervised_umap_used': best_estimator.get_was_supervised(),
			'metrics': best_estimator.get_all_performance_metrics(),
			'size_of_dataset': size_of_dataset,
			'use_only_CARD': use_only_card,
		}
		# Prepend the value 'param' to the keys in params
		for key, value in param.items():
			local_key = 'params.' + key
			output[local_key] = value
		results.append(output)

Number of experiments to run:  1
Numerical Rating
3    4469
1    2137
Name: count, dtype: int64
Numerical Rating
3    1065
1     829
Name: count, dtype: int64
run
Fitting
Getting embeddings...
Getting embeddings for batch  0  out of  1702
Getting embeddings for batch  75  out of  1702
Getting embeddings for batch  150  out of  1702
Getting embeddings for batch  225  out of  1702
Getting embeddings for batch  300  out of  1702
Getting embeddings for batch  375  out of  1702
Getting embeddings for batch  450  out of  1702
Getting embeddings for batch  525  out of  1702
Getting embeddings for batch  600  out of  1702
Getting embeddings for batch  675  out of  1702
Getting embeddings for batch  750  out of  1702
Getting embeddings for batch  825  out of  1702
Getting embeddings for batch  900  out of  1702
Getting embeddings for batch  975  out of  1702
Getting embeddings for batch  1050  out of  1702
Getting embeddings for batch  1125  out of  1702
Getting embeddings for batch  1200  out 

In [ ]:
results_df = pd.json_normalize(results)
# Get current date and time
from datetime import datetime
now = datetime.now()
results_df
results_df.to_csv(f'all_data_results_${str(now)}.csv', index=False)